In [ ]:
%pip install psycopg2-binary pandas sqlalchemy plotly ipywidgets ipympl nbformat ipython

In [1]:
def creer_dashboard_capteur(df, stats_complet, titre, nom_mesure, unite, colonne_valeur="value"):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.express as px
    import locale

    custom_palette = px.colors.qualitative.D3

    fig = make_subplots(
        rows=2, cols=2,
        column_widths=[0.4, 0.6],
        row_heights=[0.7, 0.3],
        specs=[
            [{"type": "table"}, {"type": "xy", "rowspan": 2}],
            [{"type": "table"}, None]
        ],
        horizontal_spacing=0.05,
        vertical_spacing=0.1
    )

    fig.add_trace(
        go.Table(
            header=dict(values=["Heure", f"{nom_mesure} ({unite})", "Capteur"],
                        fill_color="lightgrey", align="left"),
            cells=dict(values=[df["time_bucket"], df[colonne_valeur], df["id_tag"]],
                       fill_color="white", align="left")
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=["Capteur", f"Moyenne ({unite})", f"Min ({unite})", f"Max ({unite})", "Nb valeurs"],
                fill_color="lightgrey", align="left"
            ),
            cells=dict(
                values=[
                    stats_complet["id_tag"],
                    round(stats_complet["moyenne"], 2),
                    round(stats_complet["minimum"], 2),
                    round(stats_complet["maximum"], 2),
                    stats_complet["nb_values"]
                ],
                fill_color="white", align="left"
            )
        ),
        row=2, col=1
    )

    for i, capteur in enumerate(df["id_tag"].unique()):
        df_capteur = df[df["id_tag"] == capteur]
        color = custom_palette[i % len(custom_palette)]

        stats = stats_complet[stats_complet["id_tag"] == capteur]
        moyenne_val = stats["moyenne"].values[0]
        minimum_val = stats["minimum"].values[0]
        maximum_val = stats["maximum"].values[0]
        count_val = stats["nb_values"].values[0]

        df_capteur_custom = df_capteur.copy()
        df_capteur_custom["moyenne"] = moyenne_val
        df_capteur_custom["minimum"] = minimum_val
        df_capteur_custom["maximum"] = maximum_val
        df_capteur_custom["nb_values"] = count_val

        fig.add_trace(go.Scatter(
            x=df_capteur_custom["time_bucket"],
            y=df_capteur_custom[colonne_valeur],
            mode="lines+markers",
            name=str(capteur),
            line_shape="spline",
            customdata=df_capteur_custom[["id_tag", "moyenne", "minimum", "maximum", "nb_values"]],
            line=dict(color=color, width=2),
            marker=dict(size=3, opacity=0.7),
            hovertemplate=
                "<b>Capteur</b> : %{customdata[0]}<br>" +
                "<b>Date</b> : %{x|%d %b %Y %H:%M}<br>" +
                "<b>" + nom_mesure + "</b> : %{y:.2f} " + unite + "<br><br>" +
                "<b>Statistiques capteur</b><br>" +
                "Moyenne : %{customdata[1]:.2f} " + unite + "<br>" +
                "Min : %{customdata[2]:.2f} " + unite + "<br>" +
                "Max : %{customdata[3]:.2f} " + unite + "<br>" +
                "Nb valeurs : %{customdata[4]}<extra></extra>"
        ), row=1, col=2)

    stat_trace_indices = {}
    for idx, stat_capteur in enumerate(stats_complet["id_tag"]):
        df_ct = df if stat_capteur in ["Tous les capteurs", "global"] else df[df["id_tag"] == stat_capteur]
        x_vals = df_ct["time_bucket"]

        color = "red" if stat_capteur in ["Tous les capteurs", "global"] else custom_palette[idx % len(custom_palette)]
        gray = "grey" if stat_capteur in ["Tous les capteurs", "global"] else color

        moyenne_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "moyenne"].values[0]
        minimum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "minimum"].values[0]
        maximum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "maximum"].values[0]

        for label, val, dash in zip(["Moyenne", "Minimum", "Maximum"],
                                    [moyenne_val, minimum_val, maximum_val],
                                    ["dash", "dot", "dot"]):
            fig.add_trace(go.Scatter(
                x=x_vals, y=[val]*len(x_vals),
                mode="lines",
                name=f"{stat_capteur} - {label}",
                line=dict(dash=dash, color=color, width=1),
                visible=False,
                hovertemplate=(
                    f"<b>Capteur</b> : {stat_capteur}<br>"
                    f"<b>Type</b> : {label}<br>"
                    "<b>Valeur</b> : %{y:.2f} " + f"{unite}<extra></extra>"
                )
            ), row=1, col=2)

        stat_trace_indices[stat_capteur] = list(range(len(fig.data)-3, len(fig.data)))

    nb_base_traces = len(df["id_tag"].unique()) + 2
    buttons = [
        dict(
            label="Masquer les stats",
            method="update",
            args=[{"visible": [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)}]
        )
    ]

    for capteur in stats_complet["id_tag"]:
        visibility = [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)
        for idx in stat_trace_indices[capteur]:
            visibility[idx] = True
        buttons.append(dict(
            label=f"Stats {capteur}",
            method="update",
            args=[{"visible": visibility}]
        ))

    # 📐 Layout
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            buttons=buttons,
            direction="left",
            x=0, y=-0.3,
            xanchor="left",
            yanchor="top"
        )],
        title=titre,
        showlegend=True,
        margin=dict(l=10, r=10, t=100, b=50),
        height=850,
        yaxis=dict(
            title=f"{nom_mesure} ({unite})",
            showgrid=True,
            zeroline=False,
            tickformat="f",
            ticks="inside",
            ticklen=0,
            tickcolor="black"
        ),
        xaxis=dict(
            rangeslider=dict(visible=True),
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1 h", step="hour", stepmode="backward"),
                    dict(count=6, label="6 h", step="hour", stepmode="backward"),
                    dict(count=1, label="1 j", step="day", stepmode="backward"),
                    dict(step="all", label="Tout")
                ])
            )
        )
    )

    fig.show()

In [2]:
import pandas as pd
from sqlalchemy import create_engine

db_user = "admin"
db_password = "Changeme!1"
db_host = "localhost"
db_port = "5432"
db_name = "recorded"

connection_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

engine = create_engine(connection_url)

In [3]:
import pandas as pd
from sqlalchemy import inspect

inspector = inspect(engine)
tables = [
    t for t in inspector.get_table_names()
    if 'source_address' in [col['name'] for col in inspector.get_columns(t)]
]

union_sql = "\nUNION ALL\n".join(
    [f"SELECT source_address AS id_tag FROM {t}" for t in tables]
)
full_query = f"SELECT DISTINCT id_tag FROM (\n{union_sql}\n) AS all_sources ORDER BY id_tag;"

# Charger le résultat dans pandas
df_capteurs = pd.read_sql(full_query, engine)
df_capteurs.head()

,id_tag
0,1041420528
1,1070166865
2,1126982881
3,1523842139
4,1575827679


In [5]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_humidity.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  sensor_humidity.humidity AS value,
  sensor_humidity.source_address AS id_tag
FROM public.sensor_humidity
GROUP BY time_bucket, sensor_humidity.humidity, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.tail(10))
display(stats_complet)

,time_bucket,value,id_tag
1071,2025-07-08 10:47:11,64.73,840973111
1072,2025-07-08 10:48:06,51.43,1041420528
1073,2025-07-08 10:51:41,55.84,1126982881
1074,2025-07-08 10:56:40,50.89,1041420528
1075,2025-07-08 11:04:13,64.39,333419537
1076,2025-07-08 11:06:56,61.98,527134251
1077,2025-07-08 11:09:41,55.23,791308911
1078,2025-07-08 11:10:58,54.36,978202981
1079,2025-07-08 11:12:07,55.38,677224097
1080,2025-07-08 11:17:21,50.36,1041420528


,id_tag,moyenne,minimum,maximum,nb_values
0,1041420528,56.476804,46.46,63.79,219.0
1,1126982881,57.901439,45.10,72.45,556.0
2,1523842139,59.338000,55.61,66.14,10.0
3,2045592530,60.405000,60.11,60.70,2.0
4,23439212,63.390000,63.39,63.39,1.0
5,305822513,61.856667,61.76,62.02,3.0
6,333419537,65.846667,64.39,66.60,6.0
7,3839865,62.185000,60.19,62.73,6.0
8,527134251,58.734762,56.04,61.98,21.0
9,632461688,60.540000,59.06,61.52,6.0


In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi humidité", "Humidité", "%")

In [ ]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_pressure.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  (sensor_pressure.atmospheric_pressure / 100 ) AS value,
  sensor_pressure.source_address AS id_tag
FROM public.sensor_pressure
GROUP BY time_bucket, sensor_pressure.atmospheric_pressure, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.tail(10))
display(stats_complet)

In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi pression", "pression", "hpa")

In [ ]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_temperature.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  sensor_temperature.temperature AS value,
  sensor_temperature.source_address AS id_tag
FROM public.sensor_temperature
GROUP BY time_bucket, sensor_temperature.temperature, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.tail(10))
display(stats_complet)

In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi temperature", "temperature", "°C")